In [3]:
import pandas as pd
import numpy as np
import tellurium as te
from buildModelFromExcel import initializeValues, writeReactions

In [4]:
model_species = pd.read_excel('model_definition.xlsx', sheet_name = 'Species', engine = 'openpyxl')
model_rxns = pd.read_excel('model_definition.xlsx', sheet_name = 'Reaction', engine = 'openpyxl')


In [51]:
#check mechanism

#check if a concentration is held constant

#function to add events


In [5]:
initializeValues(model_species=model_species, model_rxns=model_rxns)

In [9]:
model = open('model.txt','r')

In [10]:
te.loada(model)

Exception: Antimony: Error in model string, line 1:  syntax error, unexpected '<'

In [91]:
#def initializeValues(model_species, model_rxns):

model_str = ''
model_str += '#Initialize concentrations \n'
for sp in model_species.iterrows():
    model_str += (sp[1]['Label'] +'=' + str(sp[1]['Starting Conc']) + '; \n')
    
model_str += '\n#Initialize kinetic values \n'
for rxn in model_rxns.iterrows():
    if not pd.isnull(model_rxns.iloc[rxn[0]]['K1']):
        #initialize value
        model_str += ('K1_'+rxn[1]['Label'] +'=' + str(rxn[1]['K1']) + '; \n')
    if not pd.isnull(model_rxns.iloc[rxn[0]]['K2']):
        model_str += ('K2_'+rxn[1]['Label'] +'=' + str(rxn[1]['K2']) + '; \n')
    if not pd.isnull(model_rxns.iloc[rxn[0]]['K2']):
        model_str += ('K3_'+rxn[1]['Label'] +'=' + str(rxn[1]['K3']) + '; \n')

with open('model.txt', 'w') as f:
    f.write(model_str)
    

In [89]:
def writeReactions():

In [ ]:
def addEvents():

In [94]:
rxn_str = '\n #Define reactions \n'
for rxn in model_rxns.iterrows():
    if rxn[1]['Mechanism'] == 'MA':
        if str(rxn[1]['Enzyme']) != 'nan':
            # mass action kinetics
            for r in rxn[1]['Reactant'].split(','):
                rxn_str += r + ' +'
            rxn_str = rxn_str[:-2] + ' + ' + rxn[1]['Enzyme'] + ' -> '
            for p in rxn[1]['Product'].split(','):
                rxn_str += p + ' +'
            rxn_str = rxn_str[:-2] + ' + ' + rxn[1]['Enzyme'] + '; '   
            rxn_str += 'K1_' + rxn[1]['Label']
            for p in rxn[1]['Product'].split(','):
                rxn_str += '*' + p 
            rxn_str += ' \n'
        else:
            for r in rxn[1]['Reactant'].split(','):
                rxn_str += r + ' +'
            rxn_str = rxn_str[:-2] + ' -> '
            for p in rxn[1]['Product'].split(','):
                rxn_str += p + ' +'
            rxn_str = rxn_str[:-2] + '; '
            rxn_str += 'K1_' + rxn[1]['Label']
            for p in rxn[1]['Product'].split(','):
                rxn_str += '*' + p 
            rxn_str += ' \n'

            

In [93]:
with open('model.txt', 'a') as f:
    f.write(rxn_str)
    

In [83]:
rxn_str = ''
for p in rxn[1]['Product'].split(','):
    rxn_str += p + ' +'
rxn_str = rxn_str[:-2] + '; '
rxn_str += 'K1_' + rxn[1]['Label']
for p in rxn[1]['Product'].split(','):
    rxn_str += '*' + p 

In [84]:
rxn_str

't3 + t3; K1_R22*t3* t3'

'#Initialize concentrations \nCO2=10.0; \nGlycine=10.0; \nSerine=2.0; \nNH3=2.0; \nPyruvic acid=2.0; \nOxaloacetic acid=2.0; \nMalic acid=2.0; \nATP=10.0; \nAMP=1.0; \nADP=1.0; \nNADH=10.0; \nNAD+=1.0; \nFdh=2.0; \nShmt=2.0; \nSda=2.0; \nPCX=2.0; \nMdh=2.0; \nPPT=2.0; \nPK=2.0; \nFdh_DNA=0.2; \nShmt_DNA=0.2; \nSda_DNA=0.2; \nPCX_DNA=0.2; \nMdh_DNA=0.2; \nPPT_DNA=0.2; \nPK_DNA=0.2; \nRNAP=6.0; \nRibosome=8.0; \nFdh_RNA=0.0; \nShmt_RNA=0.0; \nSda_RNA=0.0; \nPCX_RNA=0.0; \nMdh_RNA=0.0; \nPPT_RNA=0.0; \nPK_RNA=0.0; \n\n#Initialize kinetic values \nK1_R1=0.2; \nK1_R2=1.0; \nK1_R3=1.0; \nK1_R4=1.0; \nK1_R5=0.2; \nK1_R6=0.2; \nK1_R7=0.2; \nK1_R8=3.0; \nK1_R9=3.0; \nK1_R10=3.0; \nK1_R11=3.0; \nK1_R12=3.0; \nK1_R13=3.0; \nK1_R14=3.0; \nK1_R15=1.0; \nK1_R16=1.0; \nK1_R17=1.0; \nK1_R18=1.0; \nK1_R19=1.0; \nK1_R20=1.0; \nK1_R21=1.0; \nK1_R22=10.0; \n'